# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Trần Đàm Gia Huy
- MSSV: 20127039
- Lớp: 20CLC08

# Import

In [12]:
import pandas as pd
import numpy as np
# Import thêm dữ thư viện nếu cần
import math

# Đọc dữ liệu

In [13]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa 10 đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa 10 đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng các khác nếu cần

# Cài đặt hàm

In [14]:
# Cài đặt các hàm cần thiết ở đây
class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.inv(X.T @ X) @ X.T    # np.linalg.pinv(X)
        self.w = X_pinv @ y
        return self
    def get_params(self):
        return self.w
    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1) 

def mse(y, y_hat):
    return np.mean((y.ravel() - y_hat.ravel())**2)

def rmse(y, y_hat):
    return math.sqrt(mse(y,y_hat))
    
def FrameToNumpy(X_train, y_train, X_test, y_test):
    return X_train.to_numpy(), y_train.to_numpy().reshape(-1,1), X_test.to_numpy(), y_test.to_numpy().reshape(-1,1)



# Yêu cầu 1a: Sử dụng toàn bộ 10 đặc trưng đề bài cung cấp (2 điểm) 

In [15]:
# Phần code cho yêu cầu 1a
X_train, y_train, X_test, y_test = FrameToNumpy(X_train, y_train, X_test, y_test)
lr = OLSLinearRegression().fit(X_train, y_train)
y_test_pre = lr.predict(X_test)
w_array = lr.get_params()

# Print results
print('w values----------------')
for i in range (0,len(w_array)):
    print(f'w{i+1}: {w_array[i][0]}')

w values----------------
w1: 0.015101362735318279
w2: 0.09021998065775627
w3: 0.04292181752549435
w4: 0.13928911689488216
w5: -0.5673328270884068
w6: -0.0001007651148748953
w7: 0.7407134377587112
w8: 0.19093579767396474
w9: 24.505973591149445
w10: 2.393516607832779


In [16]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
RMSE_result = rmse(y_test, y_test_pre)
print('RMSE:', RMSE_result) 

RMSE: 7.064046430584705


Công thức hồi quy

$$\text{Life expectancy} = 0.015101*Adult Mortality + 0.090220*BMI + 0.042922*Polio + 0.139289*Diphtheria - 0.567333*HIV/AIDS - 0.000101*GDP 
    + 0.740713*Thinness age 10-19 + 0.190936*Thinness age 5-9 + 24.505974*Income composition of resources + 2.393517*Schooling$$

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng, tìm mô hình cho kết quả tốt nhất (2 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [17]:
# Phần code cho yêu cầu 1b
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu

# Initialize data
def Init_Data():
    X_train_clone, y_train_clone = X_train.copy(), y_train.copy() # Copy Data
    X_train_clone, y_train_clone = Shuffle_Data(X_train_clone, y_train_clone) # Shuffle Data
    numberSet = int(len(X_train_clone)/5)  # Get the number of data sets divided bỵ 5
    return X_train_clone, y_train_clone, numberSet

# Shuffle X_train_clone, y_train_clone
def Shuffle_Data(X_train_clone, y_train_clone):
    rand_result = np.arange(len(X_train))
    np.random.shuffle(rand_result)
    return X_train_clone[rand_result], y_train_clone[rand_result]

# 5-fold cross-validation
def cross_validation(X_train_feature, y_train_feature, numberSet):
    rmse_val = 0
    for j in range (0,5):
        lr = OLSLinearRegression()
        X_j_val = X_train_feature[numberSet*j:numberSet*(j+1),:]
        y_j_val = y_train_feature[numberSet*j:numberSet*(j+1)]
        X_j_train = np.concatenate([X_train_feature[0:numberSet*j,:], X_train_feature[numberSet*(j+1):len(X_train_feature),:]])
        y_j_train = np.concatenate([y_train_feature[0:numberSet*j], y_train_feature[numberSet*(j+1):len(y_train_feature)]])
        lr.fit(X_j_train, y_j_train)
        y_j_val_pre = lr.predict(X_j_val)
        rmse_val += rmse(y_j_val, y_j_val_pre)
    return rmse_val/5

# Execution
X_train_clone, y_train_clone, numberSet = Init_Data()
rmse_arr=np.array([cross_validation(X_train_clone[:,i:i+1], y_train_clone, numberSet) for i in range(0,10)]) # Find best feature model ( min RMSE )
bestIndex = rmse_arr.argmin() # Best index feature
X_best_feature = X_train[:,bestIndex].reshape(-1, 1) # Best feature ( min RMSE )

# Print results
for i in range (0,len(rmse_arr)):
    print(f"RMSE feature {i+1}: {rmse_arr[i]}")
print('Best feature is feature:', bestIndex+1)
print(f'Feature {bestIndex+1}', X_best_feature)


RMSE feature 1: 46.194107821647755
RMSE feature 2: 27.94027665149374
RMSE feature 3: 18.070402643853594
RMSE feature 4: 15.99193060301548
RMSE feature 5: 67.04186566087174
RMSE feature 6: 60.25639913029676
RMSE feature 7: 51.787804836618726
RMSE feature 8: 51.70905155236947
RMSE feature 9: 13.342584046061896
RMSE feature 10: 11.778949675788038
Best feature is feature: 10
Feature 10 [[ 9.9]
 [ 9.8]
 [ 9.5]
 ...
 [10. ]
 [ 9.8]
 [ 9.8]]


In [18]:
# Huấn luyện lại mô hình best_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
lr = OLSLinearRegression().fit(X_best_feature, y_train)
y_test_pre = lr.predict(X_test[:,bestIndex].reshape(-1, 1))
w = lr.get_params()
print('W:', w[0][0])


W: 5.5573993976919205


In [19]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_feature_model
RMSE_result = rmse(y_test, y_test_pre)
print('RMSE:', RMSE_result) 


RMSE: 10.26095039165537


Công thức hồi quy

$$\text{Life expectancy} = 5.5573994*Schooling$$

# Yêu cầu 1c: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

In [20]:
# Phần code cho yêu cầu 1c
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu

def first_Model_Feature(x):
    X_train_feature = np.concatenate([x[:,2:3], x[:,3:4], x[:,8:9], x[:,9:10]], axis=1)
    return X_train_feature

def second_Model_Feature(x):
    X_train_feature = np.concatenate([x[:,9:10]**3, x[:,8:9]**2, x[:,3:4]], axis=1)
    return X_train_feature

def third_Model_Feature(x):
    X_train_feature = np.concatenate([x[:,8:9]*x[:,9:10], x[:,1:2] + x[:,2:3] + x[:,3:4]], axis=1)
    return X_train_feature

X_train_feature_array = [ first_Model_Feature(X_train_clone), second_Model_Feature(X_train_clone), third_Model_Feature(X_train_clone) ]
X_test_feature_array = [ first_Model_Feature(X_test), second_Model_Feature(X_test), third_Model_Feature(X_test) ]
RMSE_array = [cross_validation(X_train_feature_array[0], y_train_clone, numberSet), cross_validation(X_train_feature_array[1], y_train_clone, numberSet), cross_validation(X_train_feature_array[2], y_train_clone, numberSet)]

bestIndex = np.array(RMSE_array).argmin() # Best index feature
X_best_feature = X_train_feature_array[bestIndex] # Best feature ( min RMSE )

for i in range (0,len(RMSE_array)):
    print(f"RMSE model {i+1}: {RMSE_array[i]}")
print('Best model is model:', bestIndex+1)
print('Best model:', X_best_feature)


RMSE model 1: 9.55407523675269
RMSE model 2: 13.200600645811434
RMSE model 3: 12.951990251649391
Best model is model: 1
Best model: [[99.    91.     0.822 17.2  ]
 [ 7.    69.     0.613 12.9  ]
 [42.    42.     0.507  9.7  ]
 ...
 [98.    98.     0.569  9.8  ]
 [78.    77.     0.668 11.   ]
 [91.    91.     0.662 12.8  ]]


In [21]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện
lr = OLSLinearRegression().fit(X_best_feature, y_train)
y_test_pre = lr.predict(X_test_feature_array[bestIndex])
w = lr.get_params()
print('W:', w)

W: [[0.14820296]
 [0.27274154]
 [5.55143115]
 [2.31214422]]


In [22]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model
RMSE_result = rmse(y_test, y_test_pre)
print('RMSE:', RMSE_result) 

RMSE: 9.692944694229913


Công thức hồi quy

$$\text{Life expectancy} = 0.14820296*Polio + 0.27274154*Diphtheria + 6.63465918*Income composition of resources + 2.23775082*Schooling$$